In [214]:
from dbtools import connect_oracle, connect_postgre, connect_server
from pathlib import Path
from datetime import datetime, time 
import pandas as pd

In [215]:
input_path = Path(r"C:\Users\jact0355\Desktop\Test\input")

In [223]:
def clean_revenue_share(df):
    # Remove leading and trailing spaces
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    # # Change date to datetime format
    # df['Start date'] = pd.to_datetime(df['Start date'], format='%Y-%m-%d')
    # df['End date'] = pd.to_datetime(df['End date'], format='%Y-%m-%d')
    # Change country, currency, payable type, invoice currency to uppercase
    df['Country'] = df['Country'].str.upper()
    df['Currency'] = df['Currency'].str.upper()
    df['Payable type'] = df['Payable type'].str.upper()
    df['Invoice Currency'] = df['Invoice Currency'].str.upper()
    # Change all Nan value in Minima per user to 0
    df['Minima per user'] = df['Minima per user'].fillna(0)
    df['Total streams'] = 0
    return df

In [232]:
def insert_revenue_share(df, file):
    connection = connect_oracle('ddrc02', 'spotify')
    with connection.cursor() as cursor:
        query = f" delete from SPOT_MON_REVENUE_TEST where file_name = '{file.name}'"
        cursor.execute(query)
        # connection.commit()
        query = f"""
                INSERT INTO SPOT_MON_REVENUE_TEST (
                    licensor, start_date, end_date, product, country
                    , currency, total_tracks, revenue_share, adjusted_user, net_revenue, 
                    minima_per_user, royalty_pool, payable_type, invoice_currency, fx_rate, file_name
                    )
                VALUES (:1, to_date(:2, 'yyyy/mm/dd'), to_date(:3, 'yyyy/mm/dd'), :4, :5, :6, :7, :8, :9, :10, :11, :12, :13, :14, :15, :16)
                """
        cursor.executemany(query, df.values.tolist())
        connection.commit()
    connection.close()

In [233]:
def main():
    for file in input_path.glob("*.txt"):
        # File name checking 
        if 'spotify-revshare-for' not in file.stem:
            print(f"Skipping {file.stem} as it is not a Spotify revenue share file")
            continue
        
        print(f"Start processing {file.stem}")
        df = pd.read_csv(file, sep="\t")
        
        print("Start data cleaning")
        df = clean_revenue_share(df)
        # return df
        print("Inserting data into database")
        # Add a new column for file name
        df['File name'] = file.name
        # return df
        # return df
        insert_revenue_share(df.values.tolist(), file)
        
        print(f"Done processing {file.stem}")

In [234]:
main()

Start processing spotify-revshare-for-filscap-202311
Start data cleaning
Inserting data into database
Connected to ORACLE spotify@ddrc02


AttributeError: 'list' object has no attribute 'values'